In [145]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [146]:
product_type = "ambient"
num_time_periods = 365

test_article = '468a73f3'

In [147]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]
# article_data = article_data[(article_data['ARTICLE_ID'] == test_article)]

In [148]:
# constants definitions
if(product_type=="frozen"):
    warehouse_volume = 50
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 900
buffer_cost = 25
default_max_order = 10000

In [149]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [150]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [151]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [152]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [153]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start='2022-06-13', end='2022-06-18', freq='D')}).astype(str)
# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

# demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
# demand = dict((date_to_index[key],value) for (key,value) in demand.items())

demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
for item in demand.keys():
    demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

#### Heuristic

In [154]:
# demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
# demand = dict((date_to_index[key],value) for (key,value) in demand.items())

#### Method 2

In [155]:
# demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
# for item in demand.keys():
#     demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

In [ ]:
def silverMeal(item):
    item_demand = list(demand[item].values())
    fixed_cost = ordering_cost_fixed[item]
    variable_cost = ordering_cost_per_tu[item]
    cu_per_tu_item = cu_per_tu[item]
    disposal_cost = clearing_cost_per_cu[item]
    life = shelf_life[item]
    # item_demand = [2,3,30,5,3,10]
    # fixed_cost = 10
    # variable_cost = 1
    # cu_per_tu_item = 1
    # life = 2
    # disposal_cost = 0
    orders = {}
    order_date = 0
    run_complete = False
    obj_val = 0
    while(not run_complete):
        min_cost = float('inf')
        orders[order_date] = item_demand[order_date]
        cum_demand = 0
        for t in range(order_date, len(item_demand)):
            cum_demand += item_demand[t]
            if(t==order_date+life):
                orders[order_date] = cum_demand
                order_date = t + 1
                break
            cost = fixed_cost + variable_cost * math.ceil(cum_demand/cu_per_tu_item)
            avg_cost = cost / (t-order_date+1)
            if(avg_cost <= min_cost):
                min_cost = avg_cost
                if(t>=len(item_demand)-1):
                    orders[order_date] = cum_demand
                    cost += ((orders[order_date]*cu_per_tu_item) - cum_demand) * disposal_cost
                    run_complete = True
                    break
            else:
                orders[order_date] = cum_demand - item_demand[t]
                cost += ((orders[order_date]*cu_per_tu_item) - cum_demand) * disposal_cost
                order_date = t
                break
        obj_val += cost
        if(order_date > len(item_demand)-1):
            break
    return (orders, obj_val)
obj_val = 0
for item in demand.keys():
    orders, val = silverMeal(item)  
    print(orders)
    obj_val += val
# print(silverMeal('00ee8964'))
print(obj_val)

{0: 128.0, 18: 117.0, 37: 65.0, 50: 76.0, 65: 73.0, 79: 37.0, 91: 90.0, 107: 78.0, 124: 52.0, 130: 116.0, 148: 50.0, 162: 62.0, 175: 61.0, 180: 208.0, 205: 51.0, 223: 24.0, 231: 143.0, 256: 26.0, 267: 19.0}
{0: 105.0, 10: 84.0, 19: 47.0, 24: 137.0, 37: 114.0, 46: 90.0, 52: 74.0, 56: 58.0, 60: 252.0, 78: 58.0, 82: 121.0, 92: 107.0, 101: 104.0, 110: 181.0, 123: 132.0, 131: 144.0, 144: 85.0, 150: 130.0, 160: 88.0, 168: 46.0, 173: 122.0, 183: 76.0, 188: 112.0, 197: 100.0, 203: 69.0, 206: 268.0, 223: 112.0, 234: 128.0, 246: 66.0, 256: 92.0, 264: 48.0, 277: 21.0}
{0: 15.0, 8: 30.0, 20: 30.0, 31: 30.0, 45: 28.0, 56: 28.0, 69: 28.0, 80: 30.0, 90: 13.0, 99: 30.0, 113: 15.0, 122: 29.0, 138: 26.0, 151: 30.0, 163: 30.0, 177: 30.0, 188: 30.0, 199: 28.0, 209: 30.0, 218: 29.0}
{0: 94.0, 11: 93.0, 24: 67.0, 34: 69.0, 46: 70.0, 56: 70.0, 76: 48.0, 90: 46.0, 100: 40.0, 109: 96.0, 124: 48.0, 133: 69.0, 149: 45.0, 161: 22.0, 169: 46.0, 183: 47.0, 195: 44.0, 205: 20.0}
{0: 72.0, 6: 148.0, 14: 175.0, 23: 13

#### Method 1

In [ ]:
class CostMatrixCell:
    def __init__(self) -> None:
        self.orders = {}
        self.order_date = None
        self.period_end = None
        self.cost = None
        self.avg_cost = None
        # self.extra_orders = []

class Order:
    def __init__(self) -> None:
        self.item = None
        self.quantity_tu = None
        self.date = None

# class ExtraOrder:
#     def __init__(self) -> None:
#         self.orders = {}
#         self.order_date = None
#         self.period_end = None


In [ ]:
n = 6
cost_matrix = [[CostMatrixCell() for __ in range(n)] for _ in range(n)]
for i in range(n):
    for j in range(n):
        if(i>j):
            cost_matrix[i][j] = None
        else:
            cost_matrix[i][j].order_date = i
            cost_matrix[i][j].period_end = j

def getOrders(cell):
    for t in range(cell.order_date, cell.period_end + 1):
        for item in demand[t].keys():
            if(item not in cell.orders.keys()):
                cell.orders[item] = 0
            cell.orders[item] += math.ceil(demand[t][item] / cu_per_tu[item])
            

def getCost(cell):
    cost = 0
    volume = 0
    for item in cell.orders.keys():
        life = shelf_life[item]
        cost += (ordering_cost_per_tu[item] * cell.orders[item]) + ordering_cost_fixed[item]
        volume += orders[item] * cu_per_tu[item] * volume_per_cu[item]
    buffer_cost = 0
    for t in range(cell.order_date,cell.period_end+1):
        if(volume > warehouse_volume):
            cost += (volume - warehouse_volume) * buffer_cost
            demand_volume = 0
            for item in demand[t].keys():
                demand_volume += volume_per_cu[item] * demand[t][item]
            volume -= demand_volume
        else:
            break
    
    cell.cost = cost
    cell.avg_cost = cost / (cell.period_end - cell.order_date + 1) 

for i in range(n):
    for j in range(n):
        if(type(cost_matrix[i][j])==CostMatrixCell):
            getCost(cost_matrix[i][j])

print(cost_matrix[0][0].cost)

# def getFirstMin(costs,first_index):
#     i = first_index
#     while(i < len(costs)-1 and costs[i].avg_cost > costs[i+1].avg_cost):
#         i = i+1
#     return (costs[i], i)

# obj_val = 0
# first_index = 0
# for i in range(n):
#     if(getFirstMin(cost_matrix[i], first_index)[1]+1 > n-1):
#         obj_val += getFirstMin(cost_matrix[i], first_index)[0].cost
#         break
#     obj_val += getFirstMin(cost_matrix[i], first_index)[0].cost
#     first_index = getFirstMin(cost_matrix[i], first_index)[1]+1

# obj_val

0


In [ ]:
n = 6
cost_matrix = [ [0] * n for _ in range(n)]
extra_orders_matrix = [ [None] * n for _ in range(n)]
for i in range(n):
    for j in range(n):
        if(i>j):
            cost_matrix[i][j] = float('inf')

# what and how much to order on day i to satisfy demand from day i to day j
def getOrders(i,j):
    orders = {}
    for t in range(i,j+1):
        for item in demand[t].keys():
            if(item not in orders.keys()):
                orders[item] = 0
            orders[item] += math.ceil(demand[t][item] / cu_per_tu[item])
    return orders

def getCost(i,j):
    orders = getOrders(i,j)
    cost = 0
    volume = 0 
    for item in orders.keys():
        life = shelf_life[item]
        cost += (ordering_cost_per_tu[item] * orders[item]) + ordering_cost_fixed[item]
        volume += orders[item] * cu_per_tu[item] * volume_per_cu[item]
    buffer_cost = 0
    for t in range(i,j+1):
        if(volume > warehouse_volume):
            cost += (volume - warehouse_volume) * buffer_cost
            demand_volume = 0
            for item in demand[t].keys():
                demand_volume += volume_per_cu[item] * demand[t][item]
            volume -= demand_volume
        else:
            break
    
    return cost, cost/(j-i+1) 

for i in range(n):
    for j in range(n):
        if(cost_matrix[i][j]!=float('inf')):
            cost_matrix[i][j] = getCost(i,j)

def getFirstMin(costs,first_index):
    i = first_index
    while(i < len(costs)-1 and costs[i][1] > costs[i+1][1]):
        i = i+1
    return (costs[i], i)

obj_val = 0
first_index = 0
for i in range(n):
    if(getFirstMin(cost_matrix[i], first_index)[1]+1 > n-1):
        obj_val += getFirstMin(cost_matrix[i], first_index)[0][0]
        break
    obj_val += getFirstMin(cost_matrix[i], first_index)[0][0]
    first_index = getFirstMin(cost_matrix[i], first_index)[1]+1

obj_val


10310.499999999996

In [ ]:
run_complete = False
obj_value = 0
order_date = 0

while(not run_complete):
    orders = {}
    min_cost = float('inf')
    for period_end in time_indexes[order_date:]:

        for item in demand[period_end].keys():
            if(item not in orders.keys()):
                orders[item] = 0
            orders[item] += math.ceil(demand[period_end][item] / cu_per_tu[item])

        cost = 0
        for item in orders:
            cost += (orders[item] * ordering_cost_per_tu[item]) + ordering_cost_fixed[item]
        avg_cost = cost / (period_end - order_date + 1)

        if(avg_cost <= min_cost):
            min_cost = avg_cost
            if(period_end==len(time_indexes)-1):
                obj_value += cost
                run_complete = True
        else:
            order_date = period_end
            break
obj_value

10310.499999999996

In [ ]:
run_complete = False
obj_value = 0
order_date = 0
while(not run_complete):
    ordered_items = set()
    min_cost = float('inf')
    for period_end in time_indexes[order_date:]:
        print(order_date, period_end)
        for item in demand[period_end].keys():
            if(demand[period_end][item] > 0):
                ordered_items.add(item)
        print(ordered_items)
        cost = 0
        for item in ordered_items:
            cost += ordering_cost_fixed[item]
        if(cost <= min_cost):
            min_cost = cost
            if(period_end==len(time_indexes)-1):
                run_complete = True
        else:
            order_date = period_end
            break
    obj_value += min_cost
obj_value

0 0
{'2cd380af', '50b96f4c', '499d4776', 'f5a9d9d9', '5280d290', 'a82fc331', 'b294630c', '189067f1', 'e270732d', '8e205b36', 'df254538', '1814bccd', '9364caed', '1c939a8e', '499ff4b3', 'd44ecf7a', '11545bca', 'e425a2dd', '781f2cb5', 'a98513e1', '982dc667', 'd0d666f6', '43a7945a', '7465367d', '327dfe80', '87bc979c', 'bb41e272', '359fe15b', '938f7d75', 'e0cfdaa5', '9718a0ea', '75a2af63', 'b43301d5', '875a176c', 'c28c3b5d', 'e71736a4', '10f2b453', '61567e8b', 'e394e81a', 'c936ca6b', '8339c92a', 'bedc5c28', '761fcc1e', '09e397f8', 'b13b307a', '020b50b0', '2410d366', 'c148f52c', '1377878b', 'c47954c7', 'b4387f22', '34ece327', '1d34848b', '44c369a4', 'd50b255e', '3038bf57', 'c9dafa61', '58499bdc', 'd4132dce', 'ba8d7c6d', '47afad28', '7f4d8743', 'a32ee9e0', 'fea8d9f0', '2d82f35e', 'd3522cdb', 'bd574fe0', '126334a5', 'c958c1bb', '68425ae4', '2f479fa5', 'e96aad87', '25b50b2a', '68311c41', '6547bcba', 'd396d0ba', 'a1136423', '097bbc43', '52c5bfab', 'b70207f6', '1e6e2355', '84293966', '1f9d7d64',

16537.4